# bloomfilter.ipynb

 This implements a Bloom filter based spell checker in python.

 Bloom filters are interesting because they are a very space efficient way
 to find if an element is a member of a set when the set can be quite large.

 False positives are possible but false negatives are not. Bloom Filters
 are explained [here](https://en.wikipedia.org/wiki/Bloom_filter).


 Basic functions:
 
  - add strings in a dictionary to the Bloom filter
  - query the Bloom filter for search strings:
             - some are known to be in the Bloom filter
             - some are known NOT to be in the Bloom filter
 This Bloom filter example uses an input file with ISO-8859-1, Latin-1.
 All strings are converted to utf-8 so that it will work with most input
 string types.

 Also, strings are terminated with \n on input.  It was decided that
 stripping off the \n was not worth time or effort so \n is included
 in the string.


In [1]:
from bitarray import bitarray

In [2]:
class BloomFilter():
    """
    A hash function takes input and outputs a unique identifier of fixed
    length which is used for identification of input.

    """

    def __init__(self, size):
        """
        Init the Bloomfilter
        Arguments:
        size - size of the bitarray

        Return:
        Ref to Bloomfilter
        """

        self.bit_array = bitarray(size)
        self.n = size
        # initialize to zero
        self.bit_array.setall(0)
        
    def insert(self, line, bf_size):
        """
        Insert into BloomFilter according to hashes
        Arguments:
        line - entry for bloomfilter
        bf_size    - size of the bloomfilter, num bits

        Return:
        index1 - first hash
        index2 - second hash
        """

        # mod to size of bitarray
        index1 = mmh3.hash((line.encode('utf-8')), 1) % bf_size
        self.bit_array[index1] = 1
        index2 = mmh3.hash((line.encode('utf-8')), 3) % bf_size
        self.bit_array[index2] = 1
        return index1, index2


In [3]:
dbg = False
items_in_filter = 100
#false negative probability
fn_prob = .10 # 10 %

In [4]:
word_list = ['A','Aasia','As','AA','AAs','AAA','AAM','AB','ABs','ABA','ABC','ABCs','ABCs','ABD','ABDs','ABM','ABMs','ABMs','ABS','AC','ACs','ACLU','ACT','ACTH','ACTHs','AD','ADs','ADC','ADD','ADP','ADPs','ADR','ADRs','AEC','AECs','AF','AFAIK','AFAIKs','AFB','AFC','AFDC','AFN','AFT','AGC','AHQ','AI','AIs','AIDS','AIDSs','AIDSes','AIs','AK','AL','ALGOL','ALGOLs','ALU','AM','AMs','AMA','AMP','AMPs','AND','ANSI','ANSIs','ANTU','ANZUS','AOL','AOLs','AP','APB','APC','APCs','APO','APR','AQ','AR','ARC','AS','ASAP','ASAT','ASCII','ASCIIs','ASCIIs','ASDIC','ASL','ASLs','ASPCA','ASSR','ATC','ATM','ATMs','ATP','ATPs','ATPase','ATPases','ATS','ATV','ATVs','AV','AWACS']

In [5]:
word_list[:10]

['A', 'Aasia', 'As', 'AA', 'AAs', 'AAA', 'AAM', 'AB', 'ABs', 'ABA']

In [6]:
search_list = ['A','B','ASL','zzz']

In [7]:
from math import log
n = items_in_filter
ln2 = log(2)  # no base means natural ln
m = int((-n * log(fn_prob)) / (ln2) ** 2)
k = (m/n) * ln2
# the number of hash functions is rounded up
num_hash = int(k+.5)
print ('A bloomfilter with {} dictionary items requires {} bitarray bits and {} hash functions'.format(n,m,num_hash))
bitarray_size = m
 

A bloomfilter with 100 dictionary items requires 479 bitarray bits and 3 hash functions


In [8]:
from bitarray import bitarray
bf = BloomFilter(bitarray_size)
print(bf.bit_array)

bitarray('00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')


In [9]:
from bitarray import bitarray
bf = BloomFilter(bitarray_size)
#print(bf.bit_array)

for i in word_list:
    myhash = hash(i)
    # selected 3 variations of the hash that was returned in order to create 3 unique indices
    index1 = int(myhash/10000000000) %bitarray_size
    index2 = int(myhash/10000) % bitarray_size
    index3 = myhash % bitarray_size
    bf.bit_array[index1] = 1
    bf.bit_array[index2] = 1
    bf.bit_array[index3] = 1

    if dbg:
            print(bf.bit_array)
            print(myhash, index1)
            print(myhash, index2)
            print(myhash, index3)
            print(' ')
        
print(bf.bit_array)

bitarray('11101010100100111101010010110010110111100101010010110001000010001110111011100101000010001011010100011101001101000110010000101100000000101010011001000010010001001101100110001100011010010001011011010011101001100010010001111101001110001111110100100101001100110000101111111110100101000111100000010000100000010010110000001100011111000001010111101010100000110010001010111010110001001011101010111101000111100011001000100100110001000101101010110001001101011100010111101001110100010011101')


In [10]:
# look for words in search_list in the bf
for i in search_list:
    myhash = hash(i)
    index1 = int(myhash/10000000000) %bitarray_size
    index2 = int(myhash/10000) % bitarray_size
    index3 = myhash % bitarray_size  
    if bf.bit_array[index1] == 0 or bf.bit_array[index2] == 0 or bf.bit_array[index3] == 0:
        print('{} was not found'.format(i))
    else:
        print('{} was found'.format(i))

A was found
B was not found
ASL was found
zzz was not found


In [11]:
print(word_list)

['A', 'Aasia', 'As', 'AA', 'AAs', 'AAA', 'AAM', 'AB', 'ABs', 'ABA', 'ABC', 'ABCs', 'ABCs', 'ABD', 'ABDs', 'ABM', 'ABMs', 'ABMs', 'ABS', 'AC', 'ACs', 'ACLU', 'ACT', 'ACTH', 'ACTHs', 'AD', 'ADs', 'ADC', 'ADD', 'ADP', 'ADPs', 'ADR', 'ADRs', 'AEC', 'AECs', 'AF', 'AFAIK', 'AFAIKs', 'AFB', 'AFC', 'AFDC', 'AFN', 'AFT', 'AGC', 'AHQ', 'AI', 'AIs', 'AIDS', 'AIDSs', 'AIDSes', 'AIs', 'AK', 'AL', 'ALGOL', 'ALGOLs', 'ALU', 'AM', 'AMs', 'AMA', 'AMP', 'AMPs', 'AND', 'ANSI', 'ANSIs', 'ANTU', 'ANZUS', 'AOL', 'AOLs', 'AP', 'APB', 'APC', 'APCs', 'APO', 'APR', 'AQ', 'AR', 'ARC', 'AS', 'ASAP', 'ASAT', 'ASCII', 'ASCIIs', 'ASCIIs', 'ASDIC', 'ASL', 'ASLs', 'ASPCA', 'ASSR', 'ATC', 'ATM', 'ATMs', 'ATP', 'ATPs', 'ATPase', 'ATPases', 'ATS', 'ATV', 'ATVs', 'AV', 'AWACS']
